In [ ]:
!pip install cloudscraper

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import csv
import time
import re
import urllib
import json
import numpy as np
import threading
from tqdm import tqdm
import pickle
import cloudscraper

#Collect link form web stack overflow

In [3]:
# beauty_soup.py

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

pages = 1
url = f"https://stackoverflow.com/questions/tagged/python?tab=votes&page={pages}&pagesize=50"
url = "https://stackoverflow.com/questions/tagged/python?tab=Newest"
scraper = cloudscraper.create_scraper(delay=10,browser="chrome")
res = scraper.get(url)
html = res.text
#page = urlopen(url)
#html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [4]:
soup.find_all('a',{'class':'s-pagination--item js-pagination-item'})

[<a class="s-pagination--item js-pagination-item" href="/questions/tagged/python?tab=newest&amp;page=2&amp;pagesize=15" rel="" title="Go to page 2">2</a>,
 <a class="s-pagination--item js-pagination-item" href="/questions/tagged/python?tab=newest&amp;page=3&amp;pagesize=15" rel="" title="Go to page 3">3</a>,
 <a class="s-pagination--item js-pagination-item" href="/questions/tagged/python?tab=newest&amp;page=4&amp;pagesize=15" rel="" title="Go to page 4">4</a>,
 <a class="s-pagination--item js-pagination-item" href="/questions/tagged/python?tab=newest&amp;page=5&amp;pagesize=15" rel="" title="Go to page 5">5</a>,
 <a class="s-pagination--item js-pagination-item" href="/questions/tagged/python?tab=newest&amp;page=145393&amp;pagesize=15" rel="" title="Go to page 145393">145393</a>,
 <a class="s-pagination--item js-pagination-item" href="/questions/tagged/python?tab=newest&amp;page=2&amp;pagesize=15" rel="next" title="Go to page 2"> Next</a>]

In [5]:
len(soup.find_all('a',{'class':'s-pagination--item js-pagination-item'})) == 0

False

In [6]:
# max_pages = int(soup.find_all('a',{'class':'s-pagination--item js-pagination-item'})[-2].text)
# # max_pages = 43654
# count_thread = 16
# num_per_thread = max_pages//16
# page_last = max_pages%16
# all_thread = [None]*count_thread
# all_link = [None]*count_thread
# all_status_link = [None]*count_thread

In [7]:
max_pages = 43645
count_thread = 80
per_vm = 8
vm_number = 0
num_per_thread = max_pages//count_thread
page_last = max_pages%count_thread
all_thread = [None]*count_thread
all_link = [None]*count_thread
all_status_link = [None]*count_thread

In [8]:
num_per_thread

545

In [9]:
page_last

45

In [10]:
# def find_all_link(all_link,start,stop,number_thread):
#     #print(f"start thread : {number_thread+1}\nstart_page : {start}\nstop_page : {stop-1}\n{'*'*20}")
#     try:
#         if all_link[number_thread] != None:
#             all_link_thread = all_link[number_thread][:start]
#         else:
#             all_link_thread = []
#         pages = start
#         for pages in range(start,stop):
#           while True:
#               url = f"https://stackoverflow.com/questions/tagged/python?tab=votes&page={pages+1}&pagesize=50"
#               while True :
#                 scraper = cloudscraper.create_scraper(delay=10,browser="chrome")
#                 res = scraper.get(url)
#                 if res.ok:
#                     break
#               html = res.text
#               #page = urlopen(url)
#               #html = page.read().decode("utf-8")
#               soup = BeautifulSoup(html, "html.parser")
#               if len(soup.find_all('h3',{'class':'s-post-summary--content-title'},{'id':'questions'})) == 0:
#                 print("sleep")
#                 time.sleep(0.1)
#               else:
#                 break
#           for c in soup.find_all('h3',{'class':'s-post-summary--content-title'},{'id':'questions'}):
#               ss = c.find_all('a',{'class':'s-link'})
#               all_link_thread.append(ss[0]["href"])
#               #print(ss)
#           if pages%50==0:
#               print(f"threading : {number_thread+1}\npage : {pages+1}\nprogress : {((pages-start+1)/(stop-start))*100:.2f}%\n{'*'*20}")
#         all_link[number_thread] = all_link_thread
#         print(f"-->threading : {number_thread+1}\npage : {pages+1} finish\n{'*'*20}")
#     except:
#         all_link[number_thread] = all_link_thread
#         print(f"-->threading : {number_thread+1}\npage : {pages+1} loop_interrup_finish\n{'*'*20}")
#         print("cont_restart")
#         find_all_link(all_link,pages,stop,number_thread)

In [11]:
def find_all_link(all_link,all_status_link,start,stop,number_thread):
    #print(f"start thread : {number_thread+1}\nstart_page : {start}\nstop_page : {stop-1}\n{'*'*20}")
    try:
        scraper = cloudscraper.create_scraper(delay=10,browser="chrome")
        if all_link[number_thread] != None:
            all_link_thread = all_link[number_thread][:start]
            all_status_link_thread = all_status_link[number_thread][:start]
        else:
            all_link_thread = []
            all_status_link_thread = []
        pages = start
        for pages in range(start,stop):
            while True:
                url = f"https://stackoverflow.com/questions/tagged/python?tab=votes&page={pages+1}&pagesize=50"
                while True :
                    res = scraper.get(url)
                    if res.ok:
                        break
                html = res.text
                #page = urlopen(url)
                #html = page.read().decode("utf-8")
                soup = BeautifulSoup(html, "html.parser")
                if len(soup.find_all('h3',{'class':'s-post-summary--content-title'},{'id':'questions'})) == 0:
                    print("sleep")
                    time.sleep(0.1)
                else:
                    break
            for c in soup.find_all('h3',{'class':'s-post-summary--content-title'},{'id':'questions'}):
                ss = c.find_all('a',{'class':'s-link'})
                all_link_thread.append(ss[0]["href"])

            for d_status in soup.find_all('div',{'class':'s-post-summary--stats js-post-summary-stats'}):
                dd_status = [status.text for status in d_status.find_all('span',{'class':'s-post-summary--stats-item-number'})]
                all_status_link_thread.append(dd_status)
            if pages%50==0:
                print(f"threading : {number_thread+1}\npage : {pages+1}\nprogress : {((pages-start+1)/(stop-start))*100:.2f}%\n{'*'*20}")
        all_link[number_thread] = all_link_thread
        all_status_link[number_thread] = all_status_link_thread
        print(f"-->threading : {number_thread+1}\npage : {pages+1} finish\n{'*'*20}")
    except:
        all_link[number_thread] = all_link_thread
        all_status_link[number_thread] = all_status_link_thread
        print(f"-->threading : {number_thread+1}\npage : {pages+1} loop_interrup_finish\n{'*'*20}")
        print("cont_restart")
        find_all_link(all_link,all_status_link,pages,stop,number_thread)

In [12]:
li = (page_last - 1)
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    if i < page_last:
        all_thread[i] = threading.Thread(target=find_all_link,args=(all_link,all_status_link,i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i))
        print(i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i)
    elif page_last == 0:
        all_thread[i] = threading.Thread(target=find_all_link,args=(all_link,all_status_link,i*num_per_thread,(i+1)*num_per_thread,i))
        print(i*num_per_thread,(i+1)*num_per_thread,i)
    else:
        all_thread[i] = threading.Thread(target=find_all_link,args=(all_link,all_status_link,i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i))
        print(i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i)

0 546 0
546 1092 1
1092 1638 2
1638 2184 3
2184 2730 4
2730 3276 5
3276 3822 6
3822 4368 7


In [13]:
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    all_thread[i].start()
for i in range(vm_number*per_vm,(vm_number+1)*per_vm):
    all_thread[i].join()

threading : 1
page : 1
progress : 0.18%
********************
threading : 2
page : 551
progress : 0.92%
********************
threading : 3
page : 1101
progress : 1.65%
********************
threading : 4
page : 1651
progress : 2.38%
********************
threading : 5
page : 2201
progress : 3.11%
********************
threading : 7
page : 3301
progress : 4.58%
********************
threading : 6
page : 2751
progress : 3.85%
********************
threading : 8
page : 3851
progress : 5.31%
********************
threading : 3
page : 1151
progress : 10.81%
********************
threading : 2
page : 601
progress : 10.07%
********************
threading : 1
page : 51
progress : 9.34%
********************
threading : 4
page : 1701
progress : 11.54%
********************
threading : 5
page : 2251
progress : 12.27%
********************
threading : 6
page : 2801
progress : 13.00%
********************
threading : 7
page : 3351
progress : 13.74%
********************
threading : 8
page : 3901
progress : 14.4

In [ ]:
# all_link[15]

In [6]:
all_link_out = []
for i in all_link:
    if i != None:
        for k in i:
            if k != None:
                all_link_out.append(k)

In [8]:
len(all_link_out)

156000

In [21]:
len(list(set(all_link_out)))

155751

In [9]:
all_link_out_array = np.array(all_link_out)

In [10]:
with open(f"link_data_stack_overflow_python_{vm_number*per_vm}_{(vm_number+1)*per_vm}.pickle", "wb") as f:
    pickle.dump(all_link_out_array, f, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
all_status_link_out = []
for i in all_status_link:
    if i != None:
        for k in i:
            if k != None:
                all_status_link_out.append(k)
            else : print("IsNone")
all_status_link_out_array = np.array(all_status_link_out)
with open(f"status_link_data_stack_overflow_python_{vm_number*per_vm}_{(vm_number+1)*per_vm}.pickle", "wb") as f:
    pickle.dump(all_status_link_out_array, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(all_status_link_out)

In [ ]:
len(list(set(all_status_link_out)))

In [ ]:
# with open("link_data_stack_overflow_python_2181255.pickle", "rb") as f:
#     save_link = pickle.load(f)

#Collect code form link

In [ ]:
with open("link_data_stack_overflow_python_2181255.pickle", "rb") as f:
    save_link = pickle.load(f)

In [ ]:
max_link = len(save_link)
max_link = 1000
count_thread = 8
num_per_thread = max_link//8
link_last = max_link%8
all_thread = [None]*count_thread
code_question = [None]*count_thread
code_answer = [None]*count_thread
webs = "https://stackoverflow.com/"
hours = 2
minute = 30

In [ ]:
def save_code_question_answer_all_data(code_question,code_answer,start,stop,number_thread):
    try:
        st = time.time()
        if (code_question[number_thread] or code_answer[number_thread]) != None:
            code_answer_in_link = code_answer[number_thread][:start]
            code_question_in_link = code_question[number_thread][:start]
        else:
            code_answer_in_link = []
            code_question_in_link = []

        replace_code = [['<code>',''],['</code>',''],['&lt;','<'],['&gt;','>'],['&amp;','&'],['&quot;','"'],['&apos;',"'"],['>>> ',''],['... ',''],['...','']]
        count = start
        for question_link in save_link[start:stop]:
            if (time.time() - st) > (hours*60 + minute)*60:
                print(f"-->threading : {number_thread+1}\nlink : {count} finish with time stop hours : {hours} minute : {minute}\n{'*'*20}")
                return 0
            url_page = webs + question_link
            #------------------------------------
            while True:
                scraper = cloudscraper.create_scraper(delay=10,browser="chrome")
                res_page = scraper.get(url)
                if res_page.ok:
                    #print(f"Thread : {number_thread} page open")
                    break
            html_page = res_page.text
            soup_page = BeautifulSoup(html_page ,features="html.parser")
            #------------------------------------
            if len(soup_page.find_all('a',{'class':'s-pagination--item'}))/2 > 1:
                last_page = soup_page.find_all('a',{'class':'s-pagination--item'})[-2].text
            else :last_page = 1
            code_question_reg = [str(soup_page.find_all('a',{'class':'question-hyperlink'})[0].text)]
            code_question_in_link.append(code_question_reg)
            answer = []
            #------------------------------------
            for page_in_question_link in range(int(last_page)):
                url_page = webs + question_link + f"?page={page_in_question_link+1}&tab=scoredesc#tab-top"
                ua = UserAgent()
                while True:
                    scraper = cloudscraper.create_scraper(delay=10,browser="chrome")
                    res_page = scraper.get(url)
                    if res_page.ok:
                        #print(f"Thread : {number_thread} page open")
                        break
                html_page = res_page.text
                soup_page = BeautifulSoup(html_page ,features="html.parser")

                answer = soup_page.find_all('div',{'id':'answers'})
                # answer = soup_page.find_all('div',{'class':'s-prose'},{'class':'js-post-body'},{'id':'answers'})
                # for i_answer in answer:
                #     for i_code in i_answer.find_all('code'):
                #         if len(str(i_code)) > 50:
                #             i_code = str(i_code)
                #             for rep in replace_code:
                #                 i_code = i_code.replace(rep[0],rep[1])
                #             # code_answer.append((((str(i_code).replace('<code>','')).replace('</code>','')).replace('&gt;&gt;&gt; ','')).replace('... ',''))
                #             code_answer.append(i_code)
                answer.append(answer[0])#*************************

            code_answer_in_link.append(answer)#**************************
            count += 1
            if count%50==0:
                print(f"threading : {number_thread+1}\nlink : {count}\nprogress : {((count-start+1)/(stop-start))*100:.2f}%\n{'*'*20}")
        code_answer[number_thread] = code_answer_in_link
        code_question[number_thread] = code_question_in_link
        print(f"-->threading : {number_thread+1}\nlink : {count} finish\n{'*'*20}")
    except:
        code_answer[number_thread] = code_answer_in_link
        code_question[number_thread] = code_question_in_link
        print(f"-->threading : {number_thread+1}\nlink : {count} loop_interrup_finish\n{'*'*20}")
        save_code_question_answer_all_data(code_question,code_answer,count,stop,number_thread)





In [ ]:
li = 0
for i in range(count_thread):
    if i < link_last:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i))
        li = i
        print(i*num_per_thread + i*(1),(i+1)*num_per_thread + (i+1)*1,i)
    elif link_last == 0:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread,(i+1)*num_per_thread,i))
        print(i*num_per_thread,(i+1)*num_per_thread,i)
    else:
        all_thread[i] = threading.Thread(target=save_code_question_answer_all_data,args=(code_question,code_answer,i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i))
        print(i*num_per_thread + (li+1)*(1),(i+1)*num_per_thread + (li+1)*1,i)

In [ ]:
for i in range(count_thread):
    all_thread[i].start()
for i in range(count_thread):
    all_thread[i].join()